In [7]:
import pickle
import numpy as np
import pandas as pd
import os
import csv
from sklearn import utils
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn import metrics
import warnings

In [8]:
def dataProduce(filename,root_path,p,init_weidu):
    with open(filename,"w") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['SUB_ID']+list(range(1,init_weidu)))
        file_list = os.listdir(root_path)
        for filename in file_list:
            file_path = root_path + "\\" + filename
            data = np.loadtxt(file_path).transpose()
            data = np.corrcoef(data)
            np.arctanh(data - np.eye(data.shape[0]))
            m, n = np.shape(data)
            # print(m,n)
            data_list = []
            for i in range(m):
                for j in range(i + 1, n):
                    data_list.append(data[i][j])
            writer.writerow([filename.split("_")[p][2:]]+data_list)

def dataInsertLabel(filename,aal_fea):
    aal_fea = list(map(str, aal_fea))
    aal_fea.insert(0, 'SUB_ID')
    raw_data = pd.read_csv(filename)
    data = pd.DataFrame(raw_data,columns=aal_fea)
    return data

def fea(aalFeaPer,aalFeaSel,aalFeaRid,p,s,r,allnum):
    feature_per = feaSel(aalFeaPer,allnum,p)
    feature_sel = feaSel(aalFeaSel,allnum,s)
    feature_rid = feaSel(aalFeaRid,allnum,r)
    features = set(feature_per) & set(feature_sel) & set(feature_rid)
    features = list(features)
    return features,len(features)

def feaSel(aalFeaPer,bigW,pinlv):
    aalFeaPer.value_counts()
    lie = list(aalFeaPer.values)
    feature_per = []
    for i in range(1, bigW+1):
        if lie[i - 1] >= pinlv:
            feature_per.append(i)
    return feature_per

In [9]:
warnings.filterwarnings("ignore")
aalFeaPer = pd.read_csv(r"F:\fmri\fmri-new\machine\multSite\res\aal\csv\aalMultPerPinlv.csv", header=None)
aalFeaSel = pd.read_csv(r"F:\fmri\fmri-new\machine\multSite\res\aal\csv\aalMultSelPinlv.csv", header=None)
aalFeaRid = pd.read_csv(r"F:\fmri\fmri-new\machine\multSite\res\aal\csv\aalMultRidPinlv.csv", header=None)
ccFeaPer = pd.read_csv(r"F:\fmri\fmri-new\machine\multSite\res\cc200\csv\cc200MultPerPinlv.csv", header=None)
ccFeaSel = pd.read_csv(r"F:\fmri\fmri-new\machine\multSite\res\cc200\csv\cc200MultSelPinlv.csv", header=None)
ccFeaRid = pd.read_csv(r"F:\fmri\fmri-new\machine\multSite\res\cc200\csv\cc200MultRidPinlv.csv", header=None)

k_fold = 10
#site_list = ['SDSU']
site_list = ['YALE','UM_2','OLIN','LEUVEN_2','SDSU']
model_path = r"F:\fmri\fmri-new\machine\multSite\modle\double"
model_list = ['log3212']
with open("result.csv", "w") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["site","modelName","test_acc","test_var","pre",
                     "pre_var","rec","rec_var","f1_sc","f1_var","auc_score","auc_var"])
    for modelName in model_list:
        num = modelName[-4:]
        aals = int(num[0])
        aalr = int(num[1])
        ccs = int(num[2])
        ccr = int(num[3])
        aal_fea, weidu0 = fea(aalFeaPer, aalFeaSel, aalFeaRid, 10, aals, aalr, 6670)
        cc200_fea, weidu1 = fea(ccFeaPer, ccFeaSel, ccFeaRid, 10, ccs, ccr, 19900)
        weidu = weidu0+weidu1
        for site in site_list:
            print("----------------------------------------------------------------------------------------")
            print(site)
            root_path = r'F:\fmri\fmri-new\machine\multSite\jiwai_test_mult\aal'+"//"+site
            filename = site+'_test_aal.csv'
            # 制作数据集
            if(site == "UM_2" or site == "LEUVEN_2"):
                dataProduce(filename,root_path,2,6671)
            else:
                dataProduce(filename,root_path,1,6671)
            root_path1 = r'F:\fmri\fmri-new\machine\multSite\jiwai_test_mult\cc200' + "//" + site
            filename1 = site + '_test_CC200.csv'
            # 制作数据集
            if (site == "UM_2" or site == "LEUVEN_2"):
                dataProduce(filename1, root_path1, 2,19901)
            else:
                dataProduce(filename1, root_path1, 1,19901)
            data0 = dataInsertLabel(filename,aal_fea)
            data1 = dataInsertLabel(filename1, cc200_fea)
            data = pd.merge(data0, data1, on='SUB_ID', how='inner')
            phenotype_data = pd.read_csv("Phenotypic_V1_0b.csv")
            label = []
            for i in data["SUB_ID"].values:
                if i in phenotype_data["SUB_ID"].values:
                    index = np.where(phenotype_data["SUB_ID"].values == i)
                    label.append(phenotype_data["DX_GROUP"].values[index][0])
            data.insert(loc=len(data.columns),column="label",value = label)

            test_data = utils.shuffle(data,random_state=0)
            test_data = test_data.drop("SUB_ID", axis=1)
            testdata_dummies = pd.get_dummies(test_data)
            test_features = testdata_dummies.iloc[:, :weidu]
            X_test = test_features.values
            y_test = testdata_dummies.iloc[:, -1].values
            print("==================================================")
            print(site,modelName)
            res = []
            test_score = []
            precision = []
            recall = []
            f1 =[]
            auc = []
            for k in range(k_fold):
                path = model_path+"//"+modelName+"//"+str(k)+".pkl"
                f = open(path,'rb')
                model = pickle.load(f)
                f.close()
                y_predict = model.predict(X_test)
                test_score.append(model.score(X_test, y_test))
                precision.append(precision_score(y_test, y_predict, average='binary'))
                recall.append(recall_score(y_test, y_predict, average='binary'))
                f1.append(f1_score(y_test, y_predict, average='binary'))
                fpr, tpr, thresholds = metrics.roc_curve(y_test, y_predict, pos_label=2)
                auc.append(metrics.auc(fpr, tpr))

            test_acc = np.mean(test_score)
            test_var = np.var(test_score)
            print("test_acc", test_acc)
            pre = np.mean(precision)
            pre_var = np.var(precision)
            print("precision", pre)
            rec = np.mean(recall)
            rec_var = np.var(recall)
            print("recall", rec)
            f1_sc = np.mean(f1)
            f1_var = np.var(f1)
            print("f1", f1_sc)
            auc_score = np.mean(auc)
            auc_var = np.var(auc)
            print('auc_score', auc_score)
            res.append(test_acc)
            res.append(test_var)
            res.append(pre)
            res.append(pre_var)
            res.append(rec)
            res.append(rec_var)
            res.append(f1_sc)
            res.append(f1_var)
            res.append(auc_score)
            res.append(auc_var)
            writer.writerow([site]+ [modelName]+ res)

----------------------------------------------------------------------------------------
YALE
YALE log3212
test_acc 0.7416666666666667
precision 0.7207613731297943
recall 0.7944444444444445
f1 0.7550351363509258
auc_score 0.7416666666666668
----------------------------------------------------------------------------------------
UM_2
UM_2 log3212
test_acc 0.759375
precision 0.6497592019263846
recall 0.8846153846153847
f1 0.748978494623656
auc_score 0.7791497975708502
----------------------------------------------------------------------------------------
OLIN
OLIN log3212
test_acc 0.7107142857142857
precision 0.7095238095238094
recall 0.7142857142857143
f1 0.7118226600985222
auc_score 0.7107142857142857
----------------------------------------------------------------------------------------
LEUVEN_2
LEUVEN_2 log3212
test_acc 0.7357142857142858
precision 0.9466666666666667
recall 0.4083333333333333
f1 0.568954248366013
auc_score 0.6947916666666666
----------------------------------------